In [27]:
import random

In [40]:
class dyadic:
    
    '''
    d = None is taken to mean d = -inf (denom = 0 or x = inf)
    '''
    def __init__(self,a:int,d,reduce=True):
        assert((d is None) or (type(d) == int))
        if reduce:
            while (a % 2) == 0:
                a //= 2
                d -= 1
        self.a = a
        self.d = d
        
        if d is None:
            self.truedenom = 0
        else:
            self.truedenom = None
            
    def __add__(self,other):
        if self.d > other.d:
            oscale = 1 << (self.d - other.d)
            sscale = 1
        elif other.d > self.d:
            oscale = 1
            sscale = 1 << (other.d - self.d)
        else:
            oscale = sscale = 1
        return dyadic(self.a*sscale + other.a*oscale,max(self.d,other.d),reduce=True)
    
    def __neg__(self):
        return dyadic(-self.a,self.d,reduce=False)
    
    def __sub__(self,other):
        return self + (-other)
    
    def __eq__(self,other):
        return (self.a == other.a) and (self.d == other.d)
    
    def __abs__(self):
        return dyadic(abs(self.a),self.d,reduce=False)
    
    def child(self,i):
        ch = dyadic((1<<i)*self.a - 1,self.d+i,reduce=False)
        #2^ia - 1 is odd, so reduction isn't necessary
        return ch
    
    
    def parent(self):
        return self + dyadic(1,self.d)
    
    def is_ancestor_of(self,other):
        if other.d < self.d:
            return False
        if other.d == self.d:
            return other.a == self.a
        
        s = self.sprev().a
        scale = 1 << (other.d - self.d)
        
        return ((2*s*scale) < other.a) and (other.a <= self.a*scale)
        
        
    
    def sprev(self):
        if self.a == 1:
            return dyadic(0,0)
        par = self.parent()
        i = self.d - par.d
        if i == 1:
            return par.sprev()
        return par.child(i-1)
        
    def __repr__(self):
        if self.truedenom is None:
            self.truedenom = 1<<self.d
        if self.truedenom == 0:
            return ("-" if self.a < 0 else "") + "INF"
        return "{}/{}".format(self.a,self.truedenom)

def ldt_inner(x,max_depth,max_degr,randomize_degr,depth=0):
    s = ''
    if depth > max_depth:
        return s
    
    s += '\t'*depth
    s += '[.' + str(x)
    s += '\n'
    
    degr = random.randint(0,max_degr)
    #print all children
    for chi in range(1,degr+1):
        ch = x.child(chi)
        s += ldt_inner(ch,max_depth,max_degr,randomize_degr,depth=depth+1)
    
    s += '\t'*depth
    s += ']\n'
    return s
    
def latex_dyadic_tree(depth,max_degr,randomize_degr=True):
    s = '\\Tree'
    x = dyadic(1,0)
    s += ldt_inner(x,depth,max_degr,randomize_degr)
    print(s)

In [41]:
latex_dyadic_tree(5,4)

\Tree[.1/1
	[.1/2
		[.1/4
			[.1/8
			]
			[.3/16
			]
			[.7/32
				[.13/64
					[.25/128
					]
					[.51/256
					]
					[.103/512
					]
					[.207/1024
					]
				]
			]
			[.15/64
				[.29/128
					[.57/256
					]
					[.115/512
					]
					[.231/1024
					]
					[.463/2048
					]
				]
				[.59/256
					[.117/512
					]
				]
				[.119/512
					[.237/1024
					]
				]
			]
		]
		[.3/8
			[.5/16
				[.9/32
					[.17/64
					]
					[.35/128
					]
					[.71/256
					]
					[.143/512
					]
				]
				[.19/64
					[.37/128
					]
					[.75/256
					]
					[.151/512
					]
					[.303/1024
					]
				]
				[.39/128
					[.77/256
					]
					[.155/512
					]
					[.311/1024
					]
				]
				[.79/256
				]
			]
		]
	]
]

